# Identifiability analysis 

This notebook does two parts of analysis:

1) Compute first-order sensitivity (Jacobian matrix)

2) Analyze sensitivities for identifiability. 

In [1]:
import pickle 
import matplotlib.pyplot as plt
import numpy as np 

## Step 1: Define parameters

In [2]:
def load_pickle(name): 
    """Used for loading data from pickle file 
    """
    file = open(name, "rb")
    data = pickle.load(file)
    file.close()
    return data
def save_pickle(data, name):
    """Used for saving data to pickle file 
    """
    file = open(name, "wb")
    pickle.dump(data, file)
    file.close()

### Define parameters

In [3]:
# parameter nominal values
parameter = {
    "C1": 2.562434e-12, 
    "hgx": 25 * 1e-3, 
    "delH_1": 98.76,
    "delH_2": 77.11, 
    "delH_3": 21.25
}

# number of parameters
n_para =len(parameter)

# parameter name list
parameter_names = list(parameter.keys())



### Define measurement names and locations

In [4]:
## Get measurement names 
measure_keys = ["y_CO2", "P", "vg", "Tg", "Ts"]

# all measurement locations
# choose Nz * Ntheta. 
# 20*20: Original discretizement used by Ryan 
# 10*10: discretizement grids chosen in this paper
# 5*5, 3*3: smaller problem size, can be used for debugging 

store_option = "10*10"

if store_option == "20*20":
    # all z locations
    z_pos = [0.01, 0.016, 0.027, 0.043, 0.071, 0.115, 0.188, 
                  0.307, 0.403, 0.5, 0.597, 0.693, 0.812, 0.885, 
                  0.929, 0.957, 0.973, 0.984, 0.99, 1.0] 
    # all theta locations
    theta_pos = [0.005, 0.008, 0.012, 0.018, 0.028, 
                      0.042, 0.065, 0.1, 0.15, 
                      0.199, 0.299, 0.349, 0.398, 0.498, 0.597, 
                      0.697, 0.796, 0.896, 0.995, 1.0]
    
elif store_option == "10*10":
    # using different z for different sections 
    # this is because for ads., 0 is fixed, for des., 1 is fixed (refer to Ryan's model)
    # z locations for ads. section
    z_pos_ads = [0.115, 0.188, 0.307, 0.403,0.5, 0.597, 0.693, 0.812, 0.885, 1.0]
    # z locations for des. section
    z_pos_des = [0, 0.115, 0.188, 0.307, 0.403,0.5, 0.597, 0.693, 0.812, 0.885]
    
    # theta locations
    theta_pos =  [0.1, 0.199, 0.299, 0.349, 0.398, 0.498, 0.597, 0.697, 0.796, 0.896]
    
elif store_option == "5*5": 
    # z locations
    z_pos = [0.01, 0.307, 0.5, 0.812, 1.0]
    # theta locations
    theta_pos = [0.005, 0.01, 0.498, 0.796, 1.0]
    
elif store_option == "3*3":
    # z locations
    z_pos = [0.01, 0.5, 1.0]
    # theta locations
    theta_pos = [0.005, 0.01, 1.0]

# number of all locations for one measurement, Nz*Ntheta
n_total_measurements = len(z_pos_ads)*len(theta_pos)


## Step 2: Compute and save Jacobian with finite difference scheme

In [5]:
def convert_tuple(file):
    """
    This function gets a dictionary, and round the fractions in the 
    keys to 3 positions 
    For e.g., {(0.1512, 0.322352): 1} is converted to {(0.151, 0.322): 1}
    
    Arguments 
    ---------
    file: dict
    
    Return 
    ------
    new_file: dict, fractions in keys are rounded to 0.001
    """
    # create dict
    new_file = {}
    # loop over each key in dict
    for a, b in file:
        # get value
        v = file[(a,b)]
        # if there is a fraction in the key
        if a not in [0,1.0]:
            # round it to 0.001 
            a = round(a, 3)
        # if b is 0.1, no need to round
        if b!= 0.1:
            b = round(b, 3)
        # create new item in the new dict
        new_file[(a,b)] = v
    return new_file

In [6]:
# For five parameters, run both its upper and lower perturbation 
# Stored ads. and des. results separately
lower_ads_c1 = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_C1low_ads")
lower_des_c1 = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_C1low_des")

lower_ads_hgx = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_hgxlow_ads")
lower_des_hgx = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_hgxlow_des")

lower_ads_h1 = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_h1low_ads")
lower_des_h1 = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_h1low_des")

lower_ads_h2 = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_h2low_ads")
lower_des_h2 = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_h2low_des")

lower_ads_h3 = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_h3low_ads")
lower_des_h3 = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_h3low_des")

# Finite difference step size
step = 0.001 

In [7]:
# we also have original case with original parameter nominal values
original_ads = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_base_ads")
original_des = load_pickle("./Countercurrent_results/MBDoE_results/Tgin_des_393/Tgin_ads_363/Tgads363_Tgdes393_base_des")

In [8]:
def gradient(upper, lower, original, step, param_value, 
             option="scale_parameter", 
             finite_diff_option="central"):
    """
    Finite difference calculation 
    
    Arguments
    ---------
    upper: float
        measurement value with forward perturbing the parameters 
    lower: float
        measurement value with backward perturbing the parameters 
    original: float
        measurement value not perturbing the parameters 
    step: float
        step size
    param_value: float
        the parameter nominal value that is perturbed 
    option: string 
        choose from "scale_parameter", "not_scale_parameter", "scale_para_and_measure"
        decide if the stepsize is scaled by the parameter nominal value
        and if the measurement is scaled by the measurement value 
    finite_diff_option: string
        choose from "upper", "lower", "central"
        upper means forward FD, lower means backward FD, central means central FD
    """
    # central finite difference
    if finite_diff_option=="central":
        if option=="scale_parameter":
            return (upper-lower)/2/step
        elif option=="not_scale_parameter":
            return (upper-lower)/2/step/param_value
        elif option=="scale_para_and_measure":
            return (upper/original-lower/original)/2/step
    # forward finite difference
    elif finite_diff_option=="upper":
        if option=="scale_parameter":
            return (upper-original)/step
        elif option=="not_scale_parameter":
            return (upper-original)/step/param_value
        elif option=="scale_para_and_measure":
            return (upper/original-1)/step
    # backward finite difference
    elif finite_diff_option=="lower":
        if option=="scale_parameter":
            return (original-lower)/step
        elif option=="not_scale_parameter":
            return (original-lower)/step/param_value
        elif option=="scale_para_and_measure":
            return (original-lower)/step
    else:
        raise ValueError("Finite_diff_option should be upper, lower, or central")

In [9]:
def jacobian_and_var(key, ads_measure=True, 
                     scale_option="scale_parameter", 
                     step=0.001):
    """
    Compute Jacobian and variance of one measurement
    
    Arguments 
    ---------
    key: measurement name 
    ads_measure: if this is this measurement in the adsorption part 
    scale_option: choose from "scale_parameter", "not_scale_parameter", "scale_para_and_measure"
        decide if the stepsize is scaled by the parameter nominal value
        and if the measurement is scaled by the measurement value 
    step: 0.001 is default 
    std: standard deviation, 0.01 is the default 
    
    Return 
    ------
    jacobian: numpy array of shape [Nm*Np], Np is # of parameters, Nm is # of total measurements  
    var: numpy array of shape [Nm]
    """
    ### Step 1: convert dict to rounded keys 
    # adsorption section measurements 
    #ads_upper_key_c1 = upper_ads_c1[key]
    ads_lower_key_c1 = convert_tuple(lower_ads_c1[key])

    #ads_upper_key_hgx = upper_ads_hgx[key]
    ads_lower_key_hgx = convert_tuple(lower_ads_hgx[key])

    #ads_upper_key_h1 = upper_ads_h1[key]
    ads_lower_key_h1 = convert_tuple(lower_ads_h1[key])

    #ads_upper_key_h2 = upper_ads_h2[key]
    ads_lower_key_h2 = convert_tuple(lower_ads_h2[key])

    #ads_upper_key_h3 = upper_ads_h3[key]
    ads_lower_key_h3 = convert_tuple(lower_ads_h3[key])

    ads_original = convert_tuple(original_ads[key])

    # desorption section measurements
    #des_upper_key_c1 = upper_des_c1[key]
    des_lower_key_c1 = convert_tuple(lower_des_c1[key])

    #des_upper_key_hgx = upper_des_hgx[key]
    des_lower_key_hgx = convert_tuple(lower_des_hgx[key])

    #des_upper_key_h1 = upper_ads_h1[key]
    des_lower_key_h1 = convert_tuple(lower_des_h1[key])

    #des_upper_key_h2 = upper_des_h2[key]
    des_lower_key_h2 = convert_tuple(lower_des_h2[key])

    #des_upper_key_h3 = upper_des_h3[key]
    des_lower_key_h3 = convert_tuple(lower_des_h3[key])

    des_original = convert_tuple(original_des[key])


    ### Step 2: Assemble Q 
    # create Q to be 2Nm*Np
    Q = np.zeros((2*n_total_measurements, n_para))
    
    # loop over z locations
    for z_idx in range(len(z_pos_ads)):
        # loop over theta locations
        for t_idx, j in enumerate(theta_pos):
            # first fill in adsorption gradients
            # current row index is z*Ntheta + t 
            row_idx = z_idx*len(theta_pos) + t_idx 
            # z location values in the original model
            i = z_pos_ads[z_idx]
            # C1 sensitivity  
            Q[row_idx][0] = gradient(ads_lower_key_c1[(i,j)], 
                                          ads_lower_key_c1[(i,j)], 
                                         ads_original[(i,j)], 
                                         step, 
                                         parameter["C1"], 
                                         option=scale_option,
                                         finite_diff_option="lower")
            
            # hgx sensitivity
            Q[row_idx][1] = gradient(ads_lower_key_hgx[(i,j)], 
                                          ads_lower_key_hgx[(i,j)], 
                                         ads_original[(i,j)], 
                                         step, 
                                         parameter["hgx"], 
                                         option=scale_option,
                                         finite_diff_option="lower")
            
            # delH_1 sensitivity 
            Q[row_idx][2] = gradient(ads_lower_key_h1[(i,j)], 
                                          ads_lower_key_h1[(i,j)], 
                                         ads_original[(i,j)], 
                                         0.0001, 
                                         parameter["delH_1"], 
                                         option=scale_option,
                                         finite_diff_option="lower")
            
            # delH_2 sensitivity
            Q[row_idx][3] = gradient(ads_lower_key_h2[(i,j)], 
                                          ads_lower_key_h2[(i,j)], 
                                         ads_original[(i,j)], 
                                         step, 
                                         parameter["delH_2"], 
                                         option=scale_option,
                                         finite_diff_option="lower")
            
            # delH_3 sensitivity
            Q[row_idx][4] = gradient(ads_lower_key_h3[(i,j)], 
                                          ads_lower_key_h3[(i,j)], 
                                         ads_original[(i,j)], 
                                         step, 
                                         parameter["delH_3"], 
                                         option=scale_option,
                                         finite_diff_option="lower")
                                            
            # then fill in adsorption gradients
            row_idx_des = n_total_measurements + row_idx
            # desorption z locations in the original model 
            i = z_pos_des[z_idx]
            # C1 sensitivity
            Q[row_idx_des][0] = gradient(des_lower_key_c1[(i,j)], 
                                          des_lower_key_c1[(i,j)], 
                                         des_original[(i,j)], 
                                         step, 
                                         parameter["C1"], 
                                         option=scale_option,
                                         finite_diff_option="lower")
            
            # hgx sensitivity
            Q[row_idx_des][1] = gradient(des_lower_key_hgx[(i,j)], 
                                        des_lower_key_hgx[(i,j)], 
                                         des_original[(i,j)], 
                                         step, 
                                         parameter["hgx"], 
                                         option=scale_option,
                                        finite_diff_option="lower")
            
            # delH_1 sensitivity
            Q[row_idx_des][2] = gradient(des_lower_key_h1[(i,j)], 
                                          des_lower_key_h1[(i,j)], 
                                         des_original[(i,j)], 
                                         0.0001, 
                                         parameter["delH_1"], 
                                         option=scale_option,
                                        finite_diff_option="lower")
            
            # delH_2 sensitivity 
            Q[row_idx_des][3] = gradient(des_lower_key_h2[(i,j)], 
                                          des_lower_key_h2[(i,j)], 
                                         des_original[(i,j)], 
                                         step, 
                                         parameter["delH_2"], 
                                         option=scale_option,
                                         finite_diff_option="lower")
            
            # delH_3 sensitivity 
            Q[row_idx_des][4] = gradient(des_lower_key_h3[(i,j)], 
                                          des_lower_key_h3[(i,j)], 
                                         des_original[(i,j)], 
                                         step, 
                                         parameter["delH_3"], 
                                         option=scale_option,
                                         finite_diff_option="lower")
    
    # variance has shape Nm*1
    variance = np.zeros((2*n_total_measurements))
    
    # Temperature has variance 1 K
    if key == "Tg" or key=="Ts":
        var = 1**2
    # Pressure variance 0.005 bar
    elif key=="P":
        var = 0.005**2
    # yCO2 variance 0.01 
    elif key=="y_CO2":
        var = 0.01**2
    # velocity variance 0.01 m/s
    elif key=="vg":
        var = 0.01**2
    # loop over measurement, give variance 
    for i in range(2*n_total_measurements):
        variance[i] = var
    
    return Q, variance
    


In [11]:
# see the Q and V for one measurement
ans_Q, ans_var = jacobian_and_var("P", 
                        scale_option="scale_parameter", 
                        step=0.001)

print(np.shape(ans_Q))
print(len(ans_var))
print(ans_Q)

# expect: 800*5, 
# ads: (z0, t0), ..., (z0, t19)
# ads: (z1, t0), ..., (z1, t19)
# ...
# des: (z0, t0), ..., (z0, t19)
# ... 
# des: (z19, t0), ..., (z19, t19)

(200, 5)
200
[[-7.11096549e-03 -1.41222179e-03  7.97172536e-01 -1.31440455e-02
  -2.22647167e-05]
 [ 3.63554892e-04 -7.38121382e-04  1.70243989e-01 -1.66418163e-03
  -1.60665157e-05]
 [-1.06976888e-03 -6.48643960e-04 -1.56065936e-02 -3.22521338e-03
  -4.48738757e-05]
 [-1.91335555e-03 -4.63101266e-04 -1.83105510e-01 -3.01765035e-03
  -5.53231161e-05]
 [-1.46994244e-03 -3.40115748e-04 -1.22225965e-01 -4.49584178e-03
  -1.41093905e-05]
 [-3.54913221e-04 -7.16451883e-04  1.13378260e+00  2.26088907e-03
   1.06112390e-05]
 [-1.85159952e-04 -5.67771507e-04  1.87650910e-01  7.46417345e-03
   9.70528480e-06]
 [-3.87652070e-05 -4.75865752e-04 -3.79921014e-02 -6.90112837e-03
   6.70088252e-06]
 [ 1.25485418e-05 -4.22258360e-04  2.69897888e-02 -7.56231396e-03
   1.03852378e-05]
 [ 5.64405500e-06 -3.69611867e-04  1.45969236e-01 -6.33562813e-03
   1.57979283e-05]
 [-1.27080599e-02 -1.15807913e-03  3.54359617e-01 -6.81488953e-03
  -3.69488866e-05]
 [ 2.42857384e-03 -1.15740742e-03  1.02165700e+00 -1

### Save Jacobian and Variance matrix

In [18]:
def save_jaco_var(jacobian, variance, store_name=None, path=None, scale_option="scale_parameter"):
    """
    Store jacobian and variance data 
    
    Arguments 
    --------
    jacobian: Jacobian array
    variance: Variance matrix array 
    store_name: file name, string 
    path: file location 
    scale_option: string, choose from "scale_parameter", "not_scale_parameter", "scale_para_and_measure"
    """
    file = open(path+store_name+"_"+scale_option, "wb")
    pickle.dump(jacobian, file)
    file.close()

    file = open(path+ "Var_"+store_name+"_"+scale_option, "wb")
    pickle.dump(variance, file)
    file.close()

In [20]:
# scale option
scale_opt = "scale_parameter"
# location option
if store_option == "20*20":
    store_path = "./jaco_and_var/"
elif store_option == "10*10":
    store_path = "./jaco_and_var_10t10/"
# go over each measurement, store to pickle
for n in measure_keys:
    ans_Q, ans_var = jacobian_and_var(n, 
                        scale_option=scale_opt, 
                        step=step)
    save_jaco_var(ans_Q, ans_var, store_name=n, path=store_path, scale_option=scale_opt)

0 0.115 0.1
100 0 0.1
1 0.115 0.199
101 0 0.199
2 0.115 0.299
102 0 0.299
3 0.115 0.349
103 0 0.349
4 0.115 0.398
104 0 0.398
5 0.115 0.498
105 0 0.498
6 0.115 0.597
106 0 0.597
7 0.115 0.697
107 0 0.697
8 0.115 0.796
108 0 0.796
9 0.115 0.896
109 0 0.896
10 0.188 0.1
110 0.115 0.1
11 0.188 0.199
111 0.115 0.199
12 0.188 0.299
112 0.115 0.299
13 0.188 0.349
113 0.115 0.349
14 0.188 0.398
114 0.115 0.398
15 0.188 0.498
115 0.115 0.498
16 0.188 0.597
116 0.115 0.597
17 0.188 0.697
117 0.115 0.697
18 0.188 0.796
118 0.115 0.796
19 0.188 0.896
119 0.115 0.896
20 0.307 0.1
120 0.188 0.1
21 0.307 0.199
121 0.188 0.199
22 0.307 0.299
122 0.188 0.299
23 0.307 0.349
123 0.188 0.349
24 0.307 0.398
124 0.188 0.398
25 0.307 0.498
125 0.188 0.498
26 0.307 0.597
126 0.188 0.597
27 0.307 0.697
127 0.188 0.697
28 0.307 0.796
128 0.188 0.796
29 0.307 0.896
129 0.188 0.896
30 0.403 0.1
130 0.307 0.1
31 0.403 0.199
131 0.307 0.199
32 0.403 0.299
132 0.307 0.299
33 0.403 0.349
133 0.307 0.349
34 0.403 0.3

## Identifiability analysis

In [21]:
def read_one_measure(key, scale_option =None):
    """
    read the Jacobian and Variance matrix of one measurement 
    
    Arguments 
    --------
    key: measurement name 
    scale_option: choose from "scale_parameter", "not_scale_parameter", "scale_para_and_measure"
    """
    jaco_file = "./jaco_and_var_10t10/"+key+"_"+scale_option
    jacobian = load_pickle(jaco_file)

    var_file = "./jaco_and_var_10t10/Var_"+key+"_"+scale_option
    var = load_pickle(var_file)
    
    return jacobian, var

In [22]:
def compute_fim(overall_jac, overall_var, z_pos_idx=[10,20,30,40], verbose=True):
    """
    compute the FIM of one measurement for given positions 
    
    Arguments 
    ---------
    overall_jac: Jacobian matrix, shape Nm*Np 
    overall_var: Variance, shape Nm*1
    z_pos_idx: the z locations included in this FIM
    verbose: if print statements
    """
    # Jacobian matrix reorganize
    jaco = np.zeros((len(z_pos_idx), n_para))
    # Var matrix reorganize
    var = np.zeros((len(z_pos_idx), len(z_pos_idx)))
    
    # loop over the chosen idx, stack Jacobian and Var
    for i in range(len(z_pos_idx)):
        var[i,i] = overall_var[z_pos_idx[i]]
        # loop over parameters
        for p in range(n_para):
            jaco[i,p] = overall_jac[z_pos_idx[i]][p]
    # Var should be inversed
    var_inv = np.linalg.inv(var)
    # compute FIM
    fim = jaco.T@var_inv@jaco
    
    if verbose:
        print("===total FIM===")
        print("log(10) determinant:", np.log10(np.linalg.det(fim)))
        print("log(10) trace:", np.log10(np.trace(fim)))
        print("eig:", np.linalg.eigvals(fim))
        print("FIM:", fim)

    return fim

In [52]:
# test to see the FIM of some measurements
test1 = list(range(80, 90))
ads_output = list(range(90, 100))
des_output = list(range(100, 110))

test2 = [0, 10, 20, 30]
chosen = [109, 119]

CCO2_jac, CCO2_var = read_one_measure("Tg", scale_option="scale_parameter")


compute_fim(CCO2_jac, CCO2_var, z_pos_idx=[25])

===total FIM===
log(10) determinant: -inf
log(10) trace: 9.218818205990042
eig: [ 2.38418579e-07  1.65507701e+09  2.57528306e-11 -1.66051939e-11
 -1.19353442e-16]
FIM: [[ 1.27489104e+00 -7.82011685e+00  4.59343070e+04  2.86745903e+02
   5.07221648e-01]
 [-7.82011685e+00  4.79681994e+01 -2.81758705e+05 -1.75888480e+03
  -3.11127182e+00]
 [ 4.59343070e+04 -2.81758705e+05  1.65501247e+09  1.03314510e+07
   1.82751892e+04]
 [ 2.86745903e+02 -1.75888480e+03  1.03314510e+07  6.44943059e+04
   1.14083263e+02]
 [ 5.07221648e-01 -3.11127182e+00  1.82751892e+04  1.14083263e+02
   2.01800619e-01]]


/tmp/ipykernel_408750/243587232.py:22: RuntimeWarning: divide by zero encountered in log10
  print("log(10) determinant:", np.log10(np.linalg.det(fim)))


array([[ 1.27489104e+00, -7.82011685e+00,  4.59343070e+04,
         2.86745903e+02,  5.07221648e-01],
       [-7.82011685e+00,  4.79681994e+01, -2.81758705e+05,
        -1.75888480e+03, -3.11127182e+00],
       [ 4.59343070e+04, -2.81758705e+05,  1.65501247e+09,
         1.03314510e+07,  1.82751892e+04],
       [ 2.86745903e+02, -1.75888480e+03,  1.03314510e+07,
         6.44943059e+04,  1.14083263e+02],
       [ 5.07221648e-01, -3.11127182e+00,  1.82751892e+04,
         1.14083263e+02,  2.01800619e-01]])

## Ranking measurements with their A-optimality value

In [24]:
measure_keys = ["vg", 
                "Tg", 
                "Ts", 
                "P",
                "y_CO2"]

In [25]:
# list for storing log10(det)
log10_det = []
# list for storing log10(trace)
log10_trace = []
# list for storing trace
original_trace= [] 
# list for storing FIMs 
fim_collection = [] 

# use a counter to go over locations for each measurement
count = 0 
# loop over measurements
for key in measure_keys:
    # read measurement Q and V
    key_jac, key_var = read_one_measure(key, scale_option="scale_parameter")
    # if measurement is velocity
    if key=="vg":
        # only get z outlet measurements, 90-110
        for j in range(90, 110):
            # compute FIM
            fim = compute_fim(key_jac, key_var, z_pos_idx=[j],verbose=False)
            fim_collection.append(fim)
            # add det, trace
            log10_det.append((np.log10(np.linalg.det(fim)), count))
            log10_trace.append((np.log10(np.trace(fim)), count))
            original_trace.append((np.trace(fim), count))
            count += 1 
    # if measurement is pressure
    elif key=="P":
        # only get inside-the-bed measurements
        for j in range(90):
            # compute FIM
            fim = compute_fim(key_jac, key_var, z_pos_idx=[j],verbose=False)
            fim_collection.append(fim)
            # add det, trace
            log10_det.append((np.log10(np.linalg.det(fim)), count))
            log10_trace.append((np.log10(np.trace(fim)), count))
            original_trace.append((np.trace(fim), count))
            count += 1 
        # only get inside-the-bed measurements
        for j in range(110,200):
            # compute FIM
            fim = compute_fim(key_jac, key_var, z_pos_idx=[j],verbose=False)
            fim_collection.append(fim)
            # add det, trace
            log10_det.append((np.log10(np.linalg.det(fim)), count))
            log10_trace.append((np.log10(np.trace(fim)), count))
            original_trace.append((np.trace(fim), count))
            count += 1 
    
    else:
        # for all other measurements, get all locations 
        for j in range(200):
            # compute FIM
            fim = compute_fim(key_jac, key_var, z_pos_idx=[j],verbose=False)
            fim_collection.append(fim)
            # add det, trace
            log10_det.append((np.log10(np.linalg.det(fim)), count))
            log10_trace.append((np.log10(np.trace(fim)), count))
            original_trace.append((np.trace(fim), count))
            count += 1 

/tmp/ipykernel_408750/2756802933.py:14: RuntimeWarning: divide by zero encountered in log10
  log10_det.append((np.log10(np.linalg.det(fim)), count))
/tmp/ipykernel_408750/2756802933.py:14: RuntimeWarning: invalid value encountered in log10
  log10_det.append((np.log10(np.linalg.det(fim)), count))
/tmp/ipykernel_408750/2756802933.py:39: RuntimeWarning: divide by zero encountered in log10
  log10_det.append((np.log10(np.linalg.det(fim)), count))
/tmp/ipykernel_408750/2756802933.py:39: RuntimeWarning: invalid value encountered in log10
  log10_det.append((np.log10(np.linalg.det(fim)), count))
/tmp/ipykernel_408750/2756802933.py:40: RuntimeWarning: divide by zero encountered in log10
  log10_trace.append((np.log10(np.trace(fim)), count))
/tmp/ipykernel_408750/2756802933.py:23: RuntimeWarning: divide by zero encountered in log10
  log10_det.append((np.log10(np.linalg.det(fim)), count))
/tmp/ipykernel_408750/2756802933.py:23: RuntimeWarning: invalid value encountered in log10
  log10_det.ap

In [27]:
# sort trace
log10_trace.sort(key=lambda x: -x[0])

In [28]:
# rank trace
trace_rank = []
for t, idx in log10_trace:
    trace_rank.append(idx)

In [29]:
# generate names of measurements, so that we can map index -- measurement and locations
measure_len = [20,200,200,180,200]
num_z_grid = 10

# a list for storing all measurement names
all_names = [] 
# loop over measurements
for i, item in enumerate(measure_keys): 
    # for vel 
    if i==0: # vg 
        # for all idx 
        for k in range(measure_len[i]):
            # get string name 
            name = [item]
            # get string for section and location
            if k < num_z_grid:
                name.append("ads")
                name.append(str(9))
                name.append(str(k))
            # get string for section and location
            else:
                k-=num_z_grid 
                name.append("des")
                name.append(str(0))
                name.append(str(k))
            # assemble 
            all_names.append("_".join(name))
    # for pressure
    elif i==3: # P 
        # for all idx   
        for k in range(measure_len[i]):
            # get string name 
            name = [item] 
            # get string for section and location
            if k < measure_len[i]//2:
                name.append("ads")
                name.append(str(k//num_z_grid))
                name.append(str(k%num_z_grid))
            # get string for section and location
            elif k >= measure_len[i]//2:
                k -= measure_len[i]//2
                k += 10 
                name.append("des")
                name.append(str(k//num_z_grid))
                name.append(str(k%num_z_grid))     
            all_names.append("_".join(name))
    # for all other measurements
    else:
        # for all idx
        for k in range(measure_len[i]):
            # get string name 
            name = [item]
            # get string for section and location
            if k < measure_len[i]//2: 
                name.append("ads")
                name.append(str(k//num_z_grid))
                name.append(str(k%num_z_grid))
            # get string for section and location
            elif k>= measure_len[i]//2:
                k -= measure_len[i]//2
                name.append("des")
                name.append(str(k//num_z_grid))
                name.append(str(k%num_z_grid))
            all_names.append("_".join(name))

In [31]:
print(trace_rank[0])
print(log10_trace[0])

245
(9.222391021347903, 245)


In [32]:
# print the rank 
for i in trace_rank:
    print(all_names[i])

Ts_ads_2_5
Tg_ads_2_5
Ts_des_9_4
Tg_des_9_4
Ts_des_8_6
Tg_des_8_6
y_CO2_des_2_3
Ts_des_5_9
Tg_des_5_9
y_CO2_des_1_3
y_CO2_des_3_3
Ts_des_7_8
Tg_des_7_8
y_CO2_des_5_9
y_CO2_des_0_3
Ts_ads_0_0
Tg_ads_0_0
y_CO2_des_4_3
Ts_ads_5_5
Tg_ads_5_5
Ts_des_9_5
Tg_des_9_5
Ts_des_6_9
Tg_des_6_9
Ts_ads_0_2
y_CO2_des_0_4
Tg_ads_0_2
y_CO2_des_6_8
Ts_ads_1_1
Tg_ads_1_1
y_CO2_des_4_9
Ts_des_8_7
Tg_des_8_7
y_CO2_des_9_4
y_CO2_des_1_4
y_CO2_des_6_2
y_CO2_des_5_3
y_CO2_des_7_8
Ts_des_4_9
Tg_des_4_9
Ts_des_7_9
Tg_des_7_9
Ts_ads_3_3
Ts_des_8_0
Tg_ads_3_3
Tg_des_8_0
Tg_des_0_6
y_CO2_des_5_2
y_CO2_des_2_4
y_CO2_des_3_9
y_CO2_des_8_6
y_CO2_des_7_2
y_CO2_des_5_8
Ts_ads_0_3
Tg_ads_0_3
Ts_ads_4_2
Ts_des_9_3
Tg_ads_4_2
Tg_des_9_3
y_CO2_des_4_2
Ts_ads_2_6
Tg_ads_2_6
Ts_ads_6_5
Tg_ads_6_5
Ts_des_8_4
Tg_des_8_4
Ts_ads_3_4
Ts_ads_5_6
Tg_ads_5_6
Tg_ads_3_4
Ts_ads_2_2
Ts_ads_6_7
Tg_ads_2_2
Tg_ads_6_7
y_CO2_des_6_9
Tg_ads_3_5
Ts_ads_3_5
Ts_des_4_3
Tg_des_4_3
Ts_des_5_8
Tg_des_5_8
y_CO2_des_2_9
Ts_des_5_3
Tg_des_5_3
Ts_des_

In [ ]:
save_pickle(all_names, "original_MO_rank") # save the rank 

### Initial solution generator 

In [40]:
def names_to_idx(names):
    """convert a measurement name to its index in the Jacobian matrix Q and Var matrix V
    
    Arguments 
    ---------
    names: list of strings 
    
    Return 
    ------
    names_idx: list of name index 
    """
    # store the idx of names
    names_idx = [] 
    # loop over names
    for n in names:
        names_idx.append(all_names.index(n))
    return names_idx

def compute_trace_from_idx(name):
    """Get trace from the calculated FIM list 
    
    Arguments
    ---------
    name: a string name of the measurement 
    """
    # convert name to idx
    sol = names_to_idx(name)
    # get trace and FIM from the list
    total_trace = sum(original_trace[i][0] for i in sol)
    total_fim = sum(fim_collection[i] for i in sol)
    
    # print results
    print(total_trace)
    print(total_fim)
    print("trace:", format(np.trace(total_fim)))
    print("det:", np.linalg.det(total_fim))
    print("log10 trace:", format(np.log10(np.trace(total_fim))))
    print("log10 det:", np.log10(np.linalg.det(total_fim)))

In [45]:
### Calculate benchmark solutions
benchmark = ["vg_ads_9_0", 
            "vg_ads_9_5", 
            #"vg_ads_9_9", 
            "vg_des_0_0", 
            "vg_des_0_5", 
            #"vg_des_0_9", 
             
            "y_CO2_ads_9_0",
             "y_CO2_ads_9_5",
             "y_CO2_ads_9_9",
             "y_CO2_des_0_0",
             "y_CO2_des_0_5",
             "y_CO2_des_0_9",
            "y_CO2_ads_9_1", 
             "y_CO2_ads_9_2",
             "y_CO2_ads_9_3",
             "y_CO2_ads_9_4",
             "y_CO2_ads_9_6",
             "y_CO2_ads_9_7",
             "y_CO2_ads_9_8", 
             #"y_CO2_des_0_1", 
             #"y_CO2_des_0_2",
             #"y_CO2_des_0_3",
             #"y_CO2_des_0_4",
             #"y_CO2_des_0_6",
             #"y_CO2_des_0_7",
             #"y_CO2_des_0_8", 
             
             
            "Tg_ads_9_0", 
            "Tg_ads_9_5", 
            "Tg_ads_9_9", 
            "Tg_des_0_0", 
            "Tg_des_0_5", 
            "Tg_des_0_9", 
            ]  


compute_trace_from_idx(benchmark)

338966985.0127491
[[ 6.60174604e+01  1.37862950e+02 -2.95673748e+04  1.15586570e+03
  -2.16367401e-01]
 [ 1.37862950e+02  4.22018787e+03 -7.99216510e+05  2.04945504e+04
  -8.03523980e+00]
 [-2.95673748e+04 -7.99216510e+05  3.38700690e+08 -8.58551533e+06
   1.27565412e+03]
 [ 1.15586570e+03  2.04945504e+04 -8.58551533e+06  2.62008742e+05
  -3.56821587e+01]
 [-2.16367401e-01 -8.03523980e+00  1.27565412e+03 -3.56821587e+01
   2.23387307e-02]]
trace: 338966985.0127491
det: 1.3099725170871796e+16
log10 trace: 8.530157400489149
log10 det: 16.11726218435608


In [ ]:
def create_initial(idx_sol):
    """create initial point for solutions 
    
    Arguments
    --------
    idx_sol: a list of index of chosen measurements 
    
    Return 
    ------
    init: a Nm*Nm matrix, where the chosen measurements are represented with 1, otherwise 0
        This can be used for a starting point for MO optimization 
    """
    # initialize the solution
    init = np.zeros((num_total_measure, num_total_measure))
    # give diagonal values
    for i in idx_sol:
        init[i,i] = 1 
    # give correlation values
    for i in range(num_total_measure):
        for j in range(num_total_measure):
            if init[i,i]==1 and init[j,j]==1:
                init[i,j] = 1 
                init[j,i] = 1 
    
    return init

sol = create_initial(names_idx)

In [ ]:
save_pickle(total_fim, "MILP_fim_8000")